<a href="https://colab.research.google.com/github/zzangdol1029/RADAR/blob/main/notebooks/log_anomaly_deeplog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#로그탐지모델(DeepLog)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import json
import os
import zipfile
import glob
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# ==================== Google Drive 마운트 ====================
from google.colab import drive
drive.mount('/content/drive')

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

# ==================== 데이터 로드 함수 (수정됨) ====================
def extract_and_load_log_data(dataset_path, zip_file_name='logFile.zip', extract_path='./extracted_logs'):
    """
    압축 파일 해제 및 로그 데이터 로드

    Args:
        dataset_path: Google Drive의 데이터셋 경로
        zip_file_name: 압축 파일 이름
        extract_path: 압축 해제 위치

    Returns:
        sessions: 로그 세션 리스트
    """
    print("=" * 50)
    print("압축 파일 해제 및 데이터 로드")
    print("=" * 50)

    # 1. 압축 파일 경로
    zip_file_path = os.path.join(dataset_path, zip_file_name)
    print(f"압축 파일: {zip_file_path}")

    # 2. 압축 해제 디렉토리 생성
    os.makedirs(extract_path, exist_ok=True)

    # 3. 압축 해제
    print(f"압축 해제 중... → {extract_path}")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # 4. 해제된 파일 목록 확인
    all_files = glob.glob(os.path.join(extract_path, '**/*'), recursive=True)
    json_files = [f for f in all_files if f.endswith('.json')]


    if json_files:
        print("\nJSON 파일 목록:")
        for f in json_files[:5]:  # 처음 5개만 출력
            print(f"  - {f}")


    # 5. 데이터 로드
    sessions = []

    # JSON 파일 로드 (배열 형태)
    for json_file in json_files:
        print(f"\n로드 중: {json_file}")
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

                # 리스트인 경우
                if isinstance(data, list):
                    sessions.extend(data)
                    print(f"  ✅ {len(data)}개 세션 로드")
                # 단일 객체인 경우
                else:
                    sessions.append(data)
                    print(f"  ✅ 1개 세션 로드")
        except Exception as e:
            print(f"  ❌ 에러: {e}")

    print(f"\n총 로드된 세션 수: {len(sessions)}")

    # 6. 데이터 샘플 출력
    if sessions:
        print("\n" + "=" * 50)
        print("데이터 샘플 (첫 번째 세션)")
        print("=" * 50)
        sample = sessions[0]
        for key, value in sample.items():
            if isinstance(value, list) and len(value) > 10:
                print(f"{key}: [{value[:5]}... ] (길이: {len(value)})")
            else:
                print(f"{key}: {value}")

    return sessions

# ==================== DeepLog 모델 ====================
class DeepLog(nn.Module):
    def __init__(self, num_classes, embedding_dim=32, hidden_size=64, num_layers=2):
        super(DeepLog, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(num_classes, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,
                           batch_first=True, dropout=0.3 if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(embedded, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# ==================== 데이터셋 클래스 ====================
class LogDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = torch.LongTensor(sequences)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

# ==================== 데이터 전처리 함수 ====================
def prepare_deeplog_data(sessions, window_size=10, stride=1, min_seq_length=5):
    """
    DeepLog 학습용 데이터 준비

    Args:
        sessions: 로그 세션 리스트
        window_size: 슬라이딩 윈도우 크기
        stride: 윈도우 이동 크기
        min_seq_length: 최소 시퀀스 길이 (이보다 짧으면 제외)
    """
    sequences = []
    labels = []
    all_event_ids = set()

    # 세션별 통계
    total_sessions = len(sessions)
    valid_sessions = 0
    skipped_sessions = 0

    for session in sessions:
        seq = session.get('event_sequence', [])

        # 빈 시퀀스 또는 너무 짧은 시퀀스 스킵
        if len(seq) < min_seq_length:
            skipped_sessions += 1
            continue

        valid_sessions += 1
        all_event_ids.update(seq)

        # 윈도우보다 작거나 같으면 스킵
        if len(seq) <= window_size:
            continue

        # Sliding Window 생성
        for i in range(0, len(seq) - window_size, stride):
            window = seq[i : i + window_size]
            target = seq[i + window_size]

            sequences.append(window)
            labels.append(target)

    print(f"\n세션 통계:")
    print(f"  - 전체 세션: {total_sessions}개")
    print(f"  - 유효 세션: {valid_sessions}개")
    print(f"  - 스킵된 세션: {skipped_sessions}개")

    # 이벤트 ID 재매핑
    event_id_map = {eid: idx+1 for idx, eid in enumerate(sorted(all_event_ids))}
    event_id_map[0] = 0  # 패딩용
    num_classes = len(event_id_map)

    # 재매핑 적용
    sequences_remapped = [[event_id_map.get(eid, 0) for eid in seq] for seq in sequences]
    labels_remapped = [event_id_map.get(label, 0) for label in labels]

    print(f"\n데이터 통계:")
    print(f"  - 총 샘플 수: {len(sequences):,}개")
    print(f"  - 윈도우 크기: {window_size}")
    print(f"  - 이벤트 클래스 수: {num_classes}")
    print(f"  - 이벤트 ID 범위: {min(all_event_ids)} ~ {max(all_event_ids)}")

    return np.array(sequences_remapped), np.array(labels_remapped), num_classes, event_id_map

# ==================== 학습 함수 ====================
def train_deeplog(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)

        outputs = model(sequences)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

def evaluate_deeplog(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for sequences, labels in test_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)

            outputs = model(sequences)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

# ==================== 메인 실행 코드 ====================
if __name__ == "__main__":
    # ========== 1단계: 데이터 로드 ==========
    dataset_path = '/content/drive/MyDrive/로그이상탐지/dataset/'
    sessions = extract_and_load_log_data(
        dataset_path=dataset_path,
        zip_file_name='logFile.zip',
        extract_path='./extracted_logs'
    )

    if not sessions:
        print("❌ 데이터를 로드할 수 없습니다!")
        exit(1)

    # ========== 2단계: 데이터 전처리 ==========
    print("\n" + "=" * 50)
    print("2단계: 데이터 전처리")
    print("=" * 50)

    window_size = 2
    sequences, labels, num_classes, event_id_map = prepare_deeplog_data(
        sessions,
        window_size=window_size,
        stride=1,
        min_seq_length=3
    )

    if len(sequences) == 0:
        print("❌ 전처리된 샘플이 없습니다! window_size를 줄여보세요.")
        exit(1)

    # ========== 3단계: Train/Test 분할 ==========
    X_train, X_test, y_train, y_test = train_test_split(
        sequences, labels, test_size=0.2, random_state=42, shuffle=True
    )

    train_dataset = LogDataset(X_train, y_train)
    test_dataset = LogDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    print(f"\n학습 데이터: {len(train_dataset):,}개")
    print(f"테스트 데이터: {len(test_dataset):,}개")

    # ========== 4단계: 모델 초기화 ==========
    print("\n" + "=" * 50)
    print("3단계: 모델 초기화")
    print("=" * 50)

    model = DeepLog(
        num_classes=num_classes,
        embedding_dim=32,
        hidden_size=64,
        num_layers=2
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    print(model)
    print(f"\n총 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

    # ========== 5단계: 학습 ==========
    print("\n" + "=" * 50)
    print("4단계: 모델 학습")
    print("=" * 50)

    num_epochs = 50
    best_acc = 0

    for epoch in range(num_epochs):
        train_loss, train_acc = train_deeplog(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = evaluate_deeplog(model, test_loader, criterion, device)

        if test_acc > best_acc:
            best_acc = test_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'num_classes': num_classes,
                'event_id_map': event_id_map,
                'window_size': window_size,
                'best_acc': best_acc
            }, 'best_deeplog_model.pth')

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}]")
            print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
            print(f"  Best Test Acc: {best_acc:.2f}%")
            print("-" * 50)

    print(f"\n✅ 학습 완료! 최고 정확도: {best_acc:.2f}%")
    print(f"📁 모델 저장 위치: best_deeplog_model.pth")

    # ========== 6단계: 이벤트 매핑 저장 ==========
    with open('event_id_mapping.json', 'w') as f:
        json.dump(event_id_map, f, indent=2)
    print(f"📁 이벤트 매핑 저장: event_id_mapping.json")

Mounted at /content/drive
사용 디바이스: cuda
압축 파일 해제 및 데이터 로드
압축 파일: /content/drive/MyDrive/로그이상탐지/dataset/logFile.zip
압축 해제 중... → ./extracted_logs

JSON 파일 목록:
  - ./extracted_logs/logFile/preprocessed_logs_2025-05-04.json
  - ./extracted_logs/logFile/preprocessed_logs_2025-05-03.json
  - ./extracted_logs/logFile/preprocessed_logs_2025-05-01.json
  - ./extracted_logs/logFile/preprocessed_logs_2025-05-02.json

로드 중: ./extracted_logs/logFile/preprocessed_logs_2025-05-04.json
  ✅ 115391개 세션 로드

로드 중: ./extracted_logs/logFile/preprocessed_logs_2025-05-03.json
  ✅ 115332개 세션 로드

로드 중: ./extracted_logs/logFile/preprocessed_logs_2025-05-01.json
  ✅ 115299개 세션 로드

로드 중: ./extracted_logs/logFile/preprocessed_logs_2025-05-02.json
  ✅ 116306개 세션 로드

총 로드된 세션 수: 462328

데이터 샘플 (첫 번째 세션)
event_sequence: [1]
has_error: False
has_warn: False
service_name: user
original_logs: ['2025-05-04 00:00:00.876 DEBUG 2044940 --- [   scheduling-1] o.h.SQL                                  :']
simplified_text: [user

1. 🏗️ 코드 구조 및 흐름 분석
이 코드는 DeepLog 알고리즘을 구현하여 로그의 이벤트 발생 순서를 학습하는 파이프라인입니다.

* 데이터 로드 (extract_and_load_log_data)

    구글 드라이브에서 logFile.zip 압축을 풉니다.

    폴더 내의 모든 .json 파일을 찾아 데이터를 읽어들입니다.


* 데이터 전처리 (prepare_deeplog_data) - 가장 중요한 부분

    Windowing: DeepLog는 "앞선 N개의 로그를 보고 다음 로그를 맞추는" 방식입니다. 여기서 window_size=10으로 설정되었습니다. (짧은 로그로 데이터량이 적어 2로 수정)

    Filtering: 너무 짧은 세션은 학습에 도움이 안 되므로 제외합니다. (min_seq_length=5)

    Mapping: 로그의 이벤트 ID(110, 200 등)를 0부터 시작하는 정수 인덱스(0, 1, 2...)로 다시 매핑합니다.

* 모델 설계 (DeepLog 클래스)

    Embedding Layer: 앞서 설명해 드린 대로 One-hot Encoding 대신 임베딩을 사용하여 효율성을 높였습니다. (embedding_dim=32)

    LSTM: 시퀀스(순서)의 흐름을 학습하는 핵심 층입니다.

    Linear: 다음에 올 이벤트가 무엇일지 확률을 계산하는 출력층입니다.

2. 📊 실행 결과 분석 (데이터 상태 진단)
실행 로그를 보면 데이터의 특성과 수정해야 할 점이 명확히 보입니다.

    ① 엄청난 데이터 손실 발생 (Critical)
* 현상:

    전체 로드된 세션: 462,328개 (매우 많음, 좋음)

    스킵된 세션: 427,287개 (약 92%가 버려짐)

    최종 학습 데이터: 724개 (딥러닝을 하기엔 턱없이 부족함)

* 원인:

    코드 설정: window_size=10, min_seq_length=5 ( 이부분도 데이터 적어서 2로 수정)

    데이터 샘플을 보면 event_sequence: [1] 처럼 길이가 1인 세션이 많습니다.

    DeepLog는 과거 10개를 봐야 하는데, 데이터 길이가 1~5밖에 안 되니 학습 데이터를 만들 수가 없어서 다 버려진 것입니다.

    ② 이벤트 종류
* 결과: 이벤트 클래스 수: 109, 이벤트 ID 범위: 1 ~ 132



In [ ]:
import torch
import torch.nn as nn
import numpy as np
import json
import os
import zipfile
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report,
    precision_recall_fscore_support, accuracy_score,
    roc_curve, auc, precision_recall_curve
)
from collections import defaultdict
import pandas as pd

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False


# ==================== 성과평가 클래스 ====================
class DeepLogEvaluator:
    """DeepLog 모델 성과평가 클래스"""

    def __init__(self, model, test_loader, device, num_classes, event_id_map):
        self.model = model
        self.test_loader = test_loader
        self.device = device
        self.num_classes = num_classes
        self.event_id_map = event_id_map
        self.reverse_map = {v: k for k, v in event_id_map.items()}

        # 평가 결과 저장
        self.predictions = []
        self.true_labels = []
        self.probabilities = []
        self.top_k_predictions = {k: [] for k in [1, 3, 5, 10]}

    def evaluate(self):
        """모델 평가 실행"""
        self.model.eval()

        print("=" * 50)
        print("모델 평가 시작")
        print("=" * 50)

        with torch.no_grad():
            for sequences, labels in self.test_loader:
                sequences = sequences.to(self.device)
                labels = labels.to(self.device)

                # 예측
                outputs = self.model(sequences)
                probs = torch.softmax(outputs, dim=1)

                # Top-1 예측
                _, predicted = outputs.max(1)

                # 저장
                self.predictions.extend(predicted.cpu().numpy())
                self.true_labels.extend(labels.cpu().numpy())
                self.probabilities.extend(probs.cpu().numpy())

                # Top-K 예측
                for k in [1, 3, 5, 10]:
                    if k <= self.num_classes:
                        _, top_k_pred = probs.topk(k, dim=1)
                        self.top_k_predictions[k].extend(top_k_pred.cpu().numpy())

        self.predictions = np.array(self.predictions)
        self.true_labels = np.array(self.true_labels)
        self.probabilities = np.array(self.probabilities)

        print(f"✅ 평가 완료! 총 {len(self.predictions):,}개 샘플")

    def calculate_top_k_accuracy(self):
        """Top-K Accuracy 계산"""
        results = {}

        for k, preds in self.top_k_predictions.items():
            if k > self.num_classes:
                continue

            preds = np.array(preds)
            correct = 0

            for i, true_label in enumerate(self.true_labels):
                if true_label in preds[i]:
                    correct += 1

            accuracy = 100.0 * correct / len(self.true_labels)
            results[f'Top-{k}'] = accuracy

        return results

    def calculate_basic_metrics(self):
        """기본 메트릭 계산"""
        accuracy = accuracy_score(self.true_labels, self.predictions) * 100
        precision, recall, f1, _ = precision_recall_fscore_support(
            self.true_labels, self.predictions, average='weighted', zero_division=0
        )

        return {
            'Accuracy': accuracy,
            'Precision': precision * 100,
            'Recall': recall * 100,
            'F1-Score': f1 * 100
        }

    def plot_confusion_matrix(self, save_path='confusion_matrix.png', max_classes=20):
        """혼동 행렬 시각화"""
        # 가장 빈번한 클래스만 표시
        unique_labels = np.unique(np.concatenate([self.true_labels, self.predictions]))

        if len(unique_labels) > max_classes:
            # 가장 빈번한 클래스 선택
            label_counts = pd.Series(self.true_labels).value_counts()
            top_labels = label_counts.head(max_classes).index.tolist()

            # 필터링
            mask = np.isin(self.true_labels, top_labels)
            filtered_true = self.true_labels[mask]
            filtered_pred = self.predictions[mask]

            cm = confusion_matrix(filtered_true, filtered_pred, labels=top_labels)
            labels = [self.reverse_map.get(l, l) for l in top_labels]
        else:
            cm = confusion_matrix(self.true_labels, self.predictions)
            labels = [self.reverse_map.get(l, l) for l in unique_labels]

        plt.figure(figsize=(12, 10))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=labels, yticklabels=labels)
        plt.title('Confusion Matrix (Top {} Classes)'.format(min(max_classes, len(unique_labels))))
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Confusion Matrix 저장: {save_path}")
        plt.close()

    def plot_top_k_accuracy(self, save_path='top_k_accuracy.png'):
        """Top-K Accuracy 시각화"""
        top_k_results = self.calculate_top_k_accuracy()

        k_values = [int(k.split('-')[1]) for k in top_k_results.keys()]
        accuracies = list(top_k_results.values())

        plt.figure(figsize=(10, 6))
        plt.plot(k_values, accuracies, marker='o', linewidth=2, markersize=8)
        plt.xlabel('K', fontsize=12)
        plt.ylabel('Accuracy (%)', fontsize=12)
        plt.title('Top-K Accuracy', fontsize=14)
        plt.grid(True, alpha=0.3)

        # 값 표시
        for k, acc in zip(k_values, accuracies):
            plt.text(k, acc + 1, f'{acc:.2f}%', ha='center', fontsize=10)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Top-K Accuracy 그래프 저장: {save_path}")
        plt.close()

        return top_k_results

    def plot_class_distribution(self, save_path='class_distribution.png'):
        """클래스 분포 시각화"""
        true_counts = pd.Series(self.true_labels).value_counts().sort_index()
        pred_counts = pd.Series(self.predictions).value_counts().sort_index()

        # 상위 20개 클래스만
        top_classes = true_counts.head(20).index

        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # 실제 분포
        axes[0].bar(range(len(top_classes)), true_counts[top_classes].values)
        axes[0].set_xlabel('Event ID')
        axes[0].set_ylabel('Count')
        axes[0].set_title('True Label Distribution (Top 20)')
        axes[0].set_xticks(range(len(top_classes)))
        axes[0].set_xticklabels([self.reverse_map.get(c, c) for c in top_classes], rotation=45)

        # 예측 분포
        axes[1].bar(range(len(top_classes)), pred_counts.reindex(top_classes, fill_value=0).values)
        axes[1].set_xlabel('Event ID')
        axes[1].set_ylabel('Count')
        axes[1].set_title('Predicted Label Distribution (Top 20)')
        axes[1].set_xticks(range(len(top_classes)))
        axes[1].set_xticklabels([self.reverse_map.get(c, c) for c in top_classes], rotation=45)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ 클래스 분포 그래프 저장: {save_path}")
        plt.close()

    def plot_probability_distribution(self, save_path='probability_distribution.png'):
        """예측 확률 분포 시각화"""
        max_probs = np.max(self.probabilities, axis=1)

        # 정답/오답 구분
        correct_mask = self.predictions == self.true_labels
        correct_probs = max_probs[correct_mask]
        wrong_probs = max_probs[~correct_mask]

        plt.figure(figsize=(12, 6))

        plt.hist(correct_probs, bins=50, alpha=0.6, label='Correct', color='green')
        plt.hist(wrong_probs, bins=50, alpha=0.6, label='Wrong', color='red')

        plt.xlabel('Maximum Probability', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.title('Prediction Probability Distribution', fontsize=14)
        plt.legend(fontsize=12)
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ 확률 분포 그래프 저장: {save_path}")
        plt.close()

    def generate_report(self, save_path='evaluation_report.txt'):
        """종합 평가 보고서 생성"""
        report = []
        report.append("=" * 70)
        report.append("DeepLog 모델 성과평가 보고서")
        report.append("=" * 70)
        report.append("")

        # 기본 메트릭
        basic_metrics = self.calculate_basic_metrics()
        report.append("1. 기본 성능 지표")
        report.append("-" * 70)
        for metric, value in basic_metrics.items():
            report.append(f"   {metric:15s}: {value:6.2f}%")
        report.append("")

        # Top-K Accuracy
        top_k_results = self.calculate_top_k_accuracy()
        report.append("2. Top-K Accuracy")
        report.append("-" * 70)
        for k, acc in top_k_results.items():
            report.append(f"   {k:10s}: {acc:6.2f}%")
        report.append("")

        # 클래스별 성능
        report.append("3. 클래스별 상세 성능 (상위 10개)")
        report.append("-" * 70)

        # 가장 빈번한 10개 클래스
        unique, counts = np.unique(self.true_labels, return_counts=True)
        top_10_classes = unique[np.argsort(-counts)][:10]

        for cls in top_10_classes:
            mask = self.true_labels == cls
            if mask.sum() == 0:
                continue

            cls_acc = accuracy_score(
                self.true_labels[mask],
                self.predictions[mask]
            ) * 100

            original_id = self.reverse_map.get(cls, cls)
            count = mask.sum()

            report.append(f"   Event {original_id:3d}: Accuracy {cls_acc:6.2f}% (샘플: {count:4d}개)")

        report.append("")
        report.append("=" * 70)

        # 파일 저장
        with open(save_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(report))

        # 콘솔 출력
        print('\n'.join(report))
        print(f"\n✅ 평가 보고서 저장: {save_path}")

        return '\n'.join(report)

# ==================== 이상 탐지 성능 평가 ====================
class AnomalyDetectionEvaluator:
    """이상 탐지 성능 평가 클래스"""

    def __init__(self, model, test_data, event_id_map, window_size, device):
        self.model = model
        self.test_data = test_data  # (sequences, labels)
        self.event_id_map = event_id_map
        self.window_size = window_size
        self.device = device
        self.reverse_map = {v: k for k, v in event_id_map.items()}

    def inject_anomalies(self, anomaly_ratio=0.3):
        """정상 데이터에 이상을 주입"""
        sequences, labels = self.test_data
        n_samples = len(sequences)
        n_anomalies = int(n_samples * anomaly_ratio)

        # 무작위로 선택
        anomaly_indices = np.random.choice(n_samples, n_anomalies, replace=False)

        # 이상 레이블 생성
        is_anomaly = np.zeros(n_samples, dtype=bool)
        is_anomaly[anomaly_indices] = True

        # 이상 데이터 생성 (레이블을 무작위 값으로 변경)
        modified_labels = labels.copy()
        num_classes = len(self.event_id_map)

        for idx in anomaly_indices:
            # 현재 레이블과 다른 무작위 레이블
            current_label = labels[idx]
            new_label = np.random.randint(1, num_classes)
            while new_label == current_label:
                new_label = np.random.randint(1, num_classes)
            modified_labels[idx] = new_label

        return sequences, modified_labels, is_anomaly

    def evaluate_anomaly_detection(self, top_k_values=[1, 3, 5]):
        """이상 탐지 성능 평가"""
        print("=" * 50)
        print("이상 탐지 성능 평가 시작")
        print("=" * 50)

        # 이상 주입
        sequences, anomaly_labels, true_anomalies = self.inject_anomalies(anomaly_ratio=0.3)

        print(f"정상 샘플: {(~true_anomalies).sum():,}개")
        print(f"이상 샘플: {true_anomalies.sum():,}개")

        results = {}

        for top_k in top_k_values:
            detected_anomalies = []

            self.model.eval()
            with torch.no_grad():
                for i in range(len(sequences)):
                    seq = torch.LongTensor([sequences[i]]).to(self.device)
                    output = self.model(seq)
                    probs = torch.softmax(output, dim=1)

                    # Top-K 예측
                    _, top_k_pred = probs.topk(top_k, dim=1)
                    top_k_pred = top_k_pred[0].cpu().numpy()

                    # 실제 레이블이 Top-K에 없으면 이상
                    actual_label = anomaly_labels[i]
                    is_detected = actual_label not in top_k_pred
                    detected_anomalies.append(is_detected)

            detected_anomalies = np.array(detected_anomalies)

            # 성능 메트릭 계산
            tp = np.sum(detected_anomalies & true_anomalies)
            fp = np.sum(detected_anomalies & ~true_anomalies)
            tn = np.sum(~detected_anomalies & ~true_anomalies)
            fn = np.sum(~detected_anomalies & true_anomalies)

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            accuracy = (tp + tn) / len(sequences)

            results[f'Top-{top_k}'] = {
                'Precision': precision * 100,
                'Recall': recall * 100,
                'F1-Score': f1 * 100,
                'Accuracy': accuracy * 100,
                'TP': tp,
                'FP': fp,
                'TN': tn,
                'FN': fn
            }

        # 결과 출력
        print("\n이상 탐지 성능:")
        print("-" * 70)
        for top_k, metrics in results.items():
            print(f"\n{top_k}:")
            print(f"  Precision: {metrics['Precision']:.2f}%")
            print(f"  Recall:    {metrics['Recall']:.2f}%")
            print(f"  F1-Score:  {metrics['F1-Score']:.2f}%")
            print(f"  Accuracy:  {metrics['Accuracy']:.2f}%")
            print(f"  TP={metrics['TP']}, FP={metrics['FP']}, TN={metrics['TN']}, FN={metrics['FN']}")

        return results

    def plot_anomaly_detection_metrics(self, results, save_path='anomaly_detection_metrics.png'):
        """이상 탐지 메트릭 시각화"""
        top_k_values = [int(k.split('-')[1]) for k in results.keys()]

        metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy']
        values = {metric: [results[f'Top-{k}'][metric] for k in top_k_values]
                  for metric in metrics}

        plt.figure(figsize=(12, 6))

        x = np.arange(len(top_k_values))
        width = 0.2

        for i, metric in enumerate(metrics):
            plt.bar(x + i * width, values[metric], width, label=metric)

        plt.xlabel('Top-K', fontsize=12)
        plt.ylabel('Score (%)', fontsize=12)
        plt.title('Anomaly Detection Performance', fontsize=14)
        plt.xticks(x + width * 1.5, [f'Top-{k}' for k in top_k_values])
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3, axis='y')

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ 이상 탐지 메트릭 그래프 저장: {save_path}")
        plt.close()

# ==================== 메인 실행 코드에 평가 추가 ====================
def train_deeplog(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)

        outputs = model(sequences)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

def evaluate_deeplog(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for sequences, labels in test_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)

            outputs = model(sequences)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

In [ ]:
# ==================== 메인 실행 ====================
if __name__ == "__main__":

    # 학습 완료 후 종합 평가 실행
    print("\n" + "=" * 50)
    print("5단계: 종합 성과평가")
    print("=" * 50)

    # 1. 기본 성능 평가
    evaluator = DeepLogEvaluator(
        model=model,
        test_loader=test_loader,
        device=device,
        num_classes=num_classes,
        event_id_map=event_id_map
    )

    # 평가 실행
    evaluator.evaluate()

    # 2. 평가 보고서 생성
    evaluator.generate_report('deeplog_evaluation_report.txt')

    # 3. 시각화
    evaluator.plot_confusion_matrix('confusion_matrix.png', max_classes=20)
    evaluator.plot_top_k_accuracy('top_k_accuracy.png')
    evaluator.plot_class_distribution('class_distribution.png')
    evaluator.plot_probability_distribution('probability_distribution.png')

    # 4. 이상 탐지 성능 평가
    anomaly_evaluator = AnomalyDetectionEvaluator(
        model=model,
        test_data=(X_test, y_test),
        event_id_map=event_id_map,
        window_size=window_size,
        device=device
    )

    anomaly_results = anomaly_evaluator.evaluate_anomaly_detection(top_k_values=[1, 3, 5])
    anomaly_evaluator.plot_anomaly_detection_metrics(anomaly_results, 'anomaly_detection_metrics.png')

    print("\n" + "=" * 50)
    print("✅ 모든 평가 완료!")
    print("=" * 50)
    print("\n생성된 파일:")
    print("  1. best_deeplog_model.pth - 학습된 모델")
    print("  2. deeplog_evaluation_report.txt - 평가 보고서")
    print("  3. confusion_matrix.png - 혼동 행렬")
    print("  4. top_k_accuracy.png - Top-K 정확도")
    print("  5. class_distribution.png - 클래스 분포")
    print("  6. probability_distribution.png - 확률 분포")
    print("  7. anomaly_detection_metrics.png - 이상 탐지 성능")


5단계: 종합 성과평가
모델 평가 시작
✅ 평가 완료! 총 23,950개 샘플
DeepLog 모델 성과평가 보고서

1. 기본 성능 지표
----------------------------------------------------------------------
   Accuracy       :  87.49%
   Precision      :  87.70%
   Recall         :  87.49%
   F1-Score       :  86.61%

2. Top-K Accuracy
----------------------------------------------------------------------
   Top-1     :  87.49%
   Top-3     :  99.39%
   Top-5     :  99.93%
   Top-10    :  99.95%

3. 클래스별 상세 성능 (상위 10개)
----------------------------------------------------------------------
   Event  34: Accuracy  95.87% (샘플: 3246개)
   Event  32: Accuracy  88.80% (샘플: 2188개)
   Event  13: Accuracy  91.73% (샘플: 2091개)
   Event  37: Accuracy  97.45% (샘플: 1842개)
   Event  90: Accuracy  97.86% (샘플: 1821개)
   Event  14: Accuracy  96.28% (샘플: 1694개)
   Event  38: Accuracy  79.86% (샘플: 1415개)
   Event  35: Accuracy  90.67% (샘플: 1158개)
   Event  91: Accuracy  96.87% (샘플: 1149개)
   Event  18: Accuracy  97.26% (샘플:  985개)


✅ 평가 보고서 저장: deeplog_evaluatio

/tmp/ipython-input-3264149151.py:346: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  seq = torch.LongTensor([sequences[i]]).to(self.device)



이상 탐지 성능:
----------------------------------------------------------------------

Top-1:
  Precision: 77.42%
  Recall:    99.93%
  F1-Score:  87.25%
  Accuracy:  91.24%
  TP=7180, FP=2094, TN=14671, FN=5

Top-3:
  Precision: 98.63%
  Recall:    98.29%
  F1-Score:  98.46%
  Accuracy:  99.08%
  TP=7062, FP=98, TN=16667, FN=123

Top-5:
  Precision: 99.78%
  Recall:    96.37%
  F1-Score:  98.05%
  Accuracy:  98.85%
  TP=6924, FP=15, TN=16750, FN=261
✅ 이상 탐지 메트릭 그래프 저장: anomaly_detection_metrics.png

✅ 모든 평가 완료!

생성된 파일:
  1. best_deeplog_model.pth - 학습된 모델
  2. deeplog_evaluation_report.txt - 평가 보고서
  3. confusion_matrix.png - 혼동 행렬
  4. top_k_accuracy.png - Top-K 정확도
  5. class_distribution.png - 클래스 분포
  6. probability_distribution.png - 확률 분포
  7. anomaly_detection_metrics.png - 이상 탐지 성능
